# Extract NDVI From Sentinel-2 Data

In [ ]:
import pystac
from planetary_computer import sign
import rasterio
import numpy as np

# Load the individual item metadata
item_url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a/items/S2A_MSIL2A_20211130T000241_R030_T55GEN_20211130T141059"
item = pystac.Item.from_file(item_url)

# Sign the item to access the assets
signed_item = sign(item)

# Get the asset hrefs for the NIR (B08) and Red (B04) bands
nir_href = signed_item.assets['B08'].href
red_href = signed_item.assets['B04'].href

# Open the NIR and Red band data
with rasterio.open(nir_href) as nir_ds, rasterio.open(red_href) as red_ds:
    nir_band = nir_ds.read(1)
    red_band = red_ds.read(1)

    # Calculate NDVI
    ndvi = (nir_band.astype(float) - red_band.astype(float)) / (nir_band + red_band)

    # Handle division by zero
    ndvi[np.isnan(ndvi)] = 0

    # Prepare output dataset properties
    kwargs = nir_ds.profile
    kwargs['driver'] = 'GTiff'
    kwargs['dtype'] = 'float32'

    output_filename = f"Gn_PAC_Total_AOI_NDVI_{signed_item.id}.tif"
    # Write the NDVI to a new file
    with rasterio.open(output_filename, 'w', **kwargs) as ndvi_ds:
        ndvi_ds.write(ndvi, 1)

In [1]:
# extract EVI 
# EVI = 2.5 \times \frac{NIR - red}{NIR + 6 \times red - 7.5 \times blue + 1}

blue_href = signed_item.assets['B02'].href

# Open the NIR and Red band data
with rasterio.open(nir_href) as nir_ds, rasterio.open(red_href) as red_ds, rasterio.open(blue_href) as blue_ds:
    nir_band = nir_ds.read(1)
    red_band = red_ds.read(1)
    blue_band = blue_ds.read(1)

    # Calculate EVI
    evi = 2.5 * ((nir_band - red_band) / (nir_band + 6 * red_band - 7.5 * blue_band + 1))
    evi[np.isnan(evi)] = 0  # Handle division by zero and invalid calculations

    # Prepare output dataset properties
    kwargs = nir_ds.profile
    kwargs['driver'] = 'GTiff'
    kwargs['dtype'] = 'float32'

    output_filename = f"Gn_PAC_Total_AOI_EVI_{signed_item.id}.tif"
    # Write the NDVI to a new file
    with rasterio.open(output_filename, 'w', **kwargs) as ndvi_ds:
        ndvi_ds.write(evi, 1)


RasterioIOError: Read or write failed. IReadBlock failed at X offset 0, Y offset 0: TIFFReadEncodedTile() failed.

In [ ]:
# extract EVI 2
# EVI = 2.5 \times \frac{NIR - red}{NIR + 6 \times red - 7.5 \times blue + 1}
# EVI2=2.5× NIR+2.4×Red+1 / NIR−Red
# Open the NIR and Red band data
# 2.5 * (NIR - RED) / ((NIR + RED + 1)
with rasterio.open(nir_href) as nir_ds, rasterio.open(red_href) as red_ds, rasterio.open(blue_href) as blue_ds:
    nir_band = nir_ds.read(1) / 10000
    red_band = red_ds.read(1) / 10000

    # Calculate EVI
    evi2 = 2.4 * ( (nir_band - red_band)  / (nir_band + red_band + 1))
    evi2[np.isnan(evi2)] = 0  # Handle division by zero and invalid calculations
    # Prepare output dataset properties
    kwargs = nir_ds.profile
    kwargs['driver'] = 'GTiff'
    kwargs['dtype'] = 'float32'

    output_filename = f"Gn_PAC_Total_AOI_EVI2_{signed_item.id}.tif"
    # Write the NDVI to a new file
    with rasterio.open(output_filename, 'w', **kwargs) as ndvi_ds:
        ndvi_ds.write(evi2, 1)